The problem gives us a 2d array of integers and we want to return the longest path that has a sequence of increasing integers. We can do a depth-first search at all possible starting points, keeping track of the maximum length and backtracking if we are unable to extend the length of the path.

DFS works by keeping a stack of nodes that are being visited. When visiting a node, each of its neighbors get pushed onto the stack and then visited.

In [1]:
nums = [
  [9,9,4],
  [6,6,8],
  [2,1,1]
]

Since we're building paths, let's build a simple path data structure. It's basically just a path represented as a list together with a length. Actually, nevermind, let's assume the python list's len implementation is O(1).

Actually, we don't need to keep a list of visited nodes because we can be sure that we will never have any cycles. We don't have cycles because the path of integers should be strictly increasing.

The neighbors function takes a cell position and returns the adjacent cell positions that are (1) in-bounds and (2) contains a larger integer value than the one in the current cell position.

In [11]:
def neighbors(xy, matrix):
    dx = [1, 0, -1, 0]
    neighbors = [(xy[0] + d[0], xy[1] + d[1]) for d in zip(dx, dx[1:] + dx[:1])]
    neighbors = filter(lambda x: x[0] >= 0 and x[0] < len(matrix) and x[1] >= 0 and x[1] < len(matrix[0]), neighbors)
    neighbors = filter(lambda x: matrix[x[0]][x[1]] > matrix[xy[0]][xy[1]], neighbors)
    return neighbors

In [16]:
neighbors((1,0), nums)

[(0, 0)]

Here's my first attempt at a recursive depth-first search visitor function. It will add the integer at the current cell position `xy` to the current path, then extend the path by branching out to all valid neighbors. It returns the longest path out of all of the neighbors.

In [24]:
def extendPath(xy, matrix, path):
    path.append(matrix[xy[0]][xy[1]])
    nbrs = neighbors(xy, matrix)
    if len(nbrs) == 0:
        return path
    else:
        max_path = path
        for n in nbrs:
            p = extendPath(n, matrix, path)
            if len(p) > len(max_path):
                max_path = p
        return max_path

In [25]:
extendPath((2, 1), nums, [])

[1, 2, 6, 9, 6, 9, 8]

I realize that the `path` parameter is passed by reference, so I fix this by making an explicit copy of the result to return. Then I pop the value from `path` as I am backtracking.

In [26]:
def extendPath(xy, matrix, path):
    path.append(matrix[xy[0]][xy[1]])
    nbrs = neighbors(xy, matrix)
    if len(nbrs) == 0:
        res = path[:]
        path.pop()
        return res
    else:
        max_path = path
        for n in nbrs:
            p = extendPath(n, matrix, path)
            if len(p) > len(max_path):
                max_path = p
        path.pop()
        return max_path

In [27]:
extendPath((2, 1), nums, [])

[1, 2, 6, 9]

So to solve the problem, I find the longest path at every possible starting cell position, then keep track of the max over the entire matrix.

In [30]:
def longestIncreasingPath(matrix):
    max_path = []
    for r in range(len(nums)):
        for c in range(len(nums[r])):
            path = extendPath((r, c), matrix, [])
            if len(path) > len(max_path):
                max_path = path
    return max_path
                
    

In [35]:
longestIncreasingPath(nums)

[1, 2, 6, 9]

In [36]:
longestIncreasingPath([
  [3,4,5],
  [3,2,6],
  [2,2,1]
])

[3, 4, 5, 6]

This algorithm can be optimized by memoizing the result at each cell position.

In [50]:
def cachePath(path, xy, cache):
    if not cache:
        return
    if xy not in cache:
        cache[xy] = path
    if len(cache[xy]) < len(path):
        cache[xy] = path
    
def extendPath(xy, matrix, path, cache = None):
    if cache and xy in cache:
        return cache[xy]
    
    val = matrix[xy[0]][xy[1]]
    nbrs = neighbors(xy, matrix)
    if len(nbrs) == 0:
        if cache is not None and xy not in cache:
            cache[xy] = [val]
        return [val]
    else:
        path.append(val)
        max_subpath = []
        for nbr in nbrs:
            subpath = extendPath(nbr, matrix, path, cache)
            if len(subpath) > len(max_subpath):
                max_subpath = subpath
        path.pop()
        if cache is not None and xy not in cache:
            cache[xy] = [val] + max_subpath
        return [val] + max_subpath

def longestIncreasingPath(matrix):
    max_path = []
    cache = {}
    for r in range(len(nums)):
        for c in range(len(nums[r])):
            path = extendPath((r, c), matrix, [], cache)
            if len(path) > len(max_path):
                max_path = path
    print cache
    return max_path

In [51]:
longestIncreasingPath([
  [3,4,5],
  [3,2,6],
  [2,2,1]
])


{(1, 2): [6], (0, 1): [4, 5, 6], (0, 0): [3, 4, 5, 6], (2, 1): [2], (1, 1): [2, 4, 5, 6], (2, 0): [2, 3], (2, 2): [1, 2], (1, 0): [3], (0, 2): [5, 6]}


[3, 4, 5, 6]

So some comments about what I wrote.

I didn't use the `cachePath` function at all.

The `path` parameter to `extendPath` served no function.

I cannot use the C++ idiom of checking whether something is null like so
```
cache = {}
if not cache:
    ...
```
because an empty cache also evaluates to False. So I need to do something more verbose and explicitly check whether `cache is None`.

In [59]:
def cachePath(path, xy, cache):
    if cache is None:
        return
    if xy not in cache:
        cache[xy] = path
    if len(cache[xy]) < len(path):
        cache[xy] = path
    
def extendPath(xy, matrix, cache = None):
    if cache and xy in cache:
        return cache[xy]
    
    val = matrix[xy[0]][xy[1]]
    nbrs = neighbors(xy, matrix)
    if len(nbrs) == 0:
        cachePath([val], xy, cache)
        return [val]
    else:
        max_subpath = []
        for nbr in nbrs:
            subpath = extendPath(nbr, matrix, cache)
            if len(subpath) > len(max_subpath):
                max_subpath = subpath
        cachePath([val] + max_subpath, xy, cache)
        return [val] + max_subpath

def longestIncreasingPath(matrix):
    max_path = []
    cache = {}
    for r in range(len(nums)):
        for c in range(len(nums[r])):
            path = extendPath((r, c), matrix, cache)
            if len(path) > len(max_path):
                max_path = path
    print cache
    return max_path

In [60]:
longestIncreasingPath([
  [3,4,5],
  [3,2,6],
  [2,2,1]
])

{(1, 2): [6], (0, 1): [4, 5, 6], (0, 0): [3, 4, 5, 6], (2, 1): [2], (1, 1): [2, 4, 5, 6], (2, 0): [2, 3], (2, 2): [1, 2], (1, 0): [3], (0, 2): [5, 6]}


[3, 4, 5, 6]

In [63]:
class Solution(object):
    def neighbors(self, xy, matrix):
        dx = [1, 0, -1, 0]
        neighbors = [(xy[0] + d[0], xy[1] + d[1]) for d in zip(dx, dx[1:] + dx[:1])]
        neighbors = filter(lambda x: x[0] >= 0 and x[0] < len(matrix) and x[1] >= 0 and x[1] < len(matrix[0]), neighbors)
        neighbors = filter(lambda x: matrix[x[0]][x[1]] > matrix[xy[0]][xy[1]], neighbors)
        return neighbors

    def cachePath(self, path, xy, cache):
        if cache is None:
            return
        if xy not in cache:
            cache[xy] = path
        if len(cache[xy]) < len(path):
            cache[xy] = path

    def extendPath(self, xy, matrix, cache = None):
        if cache and xy in cache:
            return cache[xy]

        val = matrix[xy[0]][xy[1]]
        nbrs = self.neighbors(xy, matrix)
        if len(nbrs) == 0:
            self.cachePath([val], xy, cache)
            return [val]
        else:
            max_subpath = []
            for nbr in nbrs:
                subpath = self.extendPath(nbr, matrix, cache)
                if len(subpath) > len(max_subpath):
                    max_subpath = subpath
            self.cachePath([val] + max_subpath, xy, cache)
            return [val] + max_subpath

    def longestIncreasingPath(self, matrix):
        max_path = []
        cache = {}
        for r in range(len(nums)):
            for c in range(len(nums[r])):
                path = self.extendPath((r, c), matrix, cache)
                if len(path) > len(max_path):
                    max_path = path
        return max_path

In [64]:
soln = Solution()
soln.longestIncreasingPath([
  [3,4,5],
  [3,2,6],
  [2,2,1]
])

[3, 4, 5, 6]

This is my submission to the online judge, it is pretty slow at 1552 ms.
![Slow](img/329-runtime.png)